### Importações de bibliotecas

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

import MetaTrader5 as mt

import warnings
warnings.filterwarnings('ignore') # Ignorar os warnings do código

from time import sleep

# Definição das variáveis de entrada
ativo = 'VALE3'
lote = 100 # minimo de 100 para ações (1 para mercado fracionário)
tf = mt.TIMEFRAME_M5 #timeframe do gráfico

# Ler os dados dentro de um data frame         dropar algumas colunas, considerando apenas OHLC
df = pd.read_csv('./Data/VALE3_D1_2017_2020_data.csv').drop(columns = ['TickVol', 'Vol', 'Spread'])

df['Date'] = pd.to_datetime(df['Date'])

df['Adj Close'] = df['Close']

### Estratégia usando uma média simples

In [ ]:
def estrategia():
    y_data = pd.DataFrame()

    X_data = df[['Open', 'High', 'Low', 'Close']]
    y_data['Close'] = df['Close'].shift(-1)

    trade_logic = []

    for i in range(len(X_data['Close'])):
        if X_data['Close'][i] > y_data['Close'][i]:
            trade_logic.append(1)
        elif X_data['Close'][i] < y_data['Close'][i]:
            trade_logic.append(0)
        else:
            trade_logic.append(-1)

    X_data['Signal'] = trade_logic

    y = X_data['Signal']
    X = X_data.drop('Signal', axis=1)

### Definir variáveis de treino e de teste

In [ ]:
# 75% para teste e os outros 25% para treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.75, random_state=42)

### Definindo o modelo polinomial

In [ ]:
def modelo():
    # Ordem do modelo
    pol_reg = PolynomialFeatures(degree=2)
    X_pol = pol_reg.fit_transform(X_train, y_train)

    lin_reg = LinearRegression()
    lin_reg.fit(X_pol, y_train)
    y_pred = lin_reg.predict(X_pol)

### Conexão com o MetaTrader5

In [ ]:
def conexao():
    mt.initialize()
    print('Connected')

    rates = mt.copy_rates_from_pos(ativo, tf, 0, 20)
    print('Copying the values...')

    mt.shutdown()
    print('Desconnected')

    open_price  = rates[0][1]  # Open
    high_price  = rates[0][2]  # High
    low_price   = rates[0][3]  # Low
    close_price = rates[0][4]  # Close

### Definindo a função de Compra

In [ ]:
def buy():
    price = mt.symbol_info_tick(ativo).ask

    sl  = price - 200.0*mt.symbol_info(ativo).point
    tp  = price + 200.0*mt.symbol_info(ativo).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ativo,
        "volume": lote,
        "type": mt.ORDER_TYPE_BUY,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Buy Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)

    print(f'OrderSended buy: {result}')

### Definindo a função de Venda

In [ ]:
def sell():
    price = mt.symbol_info_tick(ativo).bid
    
    sl = price + 200.0*mt.symbol_info(ativo).point
    tp = price - 200.0*mt.symbol_info(ativo).point

    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": ativo,
        "volume": lote,
        "type": mt.ORDER_TYPE_SELL,
        "sl": sl,
        "tp": tp,
        "magic": 124512,
        "deviation": 0,
        "comment": "Sell Order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_FOK,
    }

    result = mt.order_send(request)
    
    print(f'OrderSended sell: {result}')

### Efetuando as ordens de compra e de venda a mercado

In [ ]:
def operacao():
    prices = [open_price, high_price, low_price, close_price]

    # 5 segundos
    tempo = 5

    while True:
        sleep(tempo)
        predict_result = lin_reg.predict([prices])

        mt.initialize()

        if mt.positions_get(symbol = ativo) == ():
            if predict_result[-1] == 1:
                print('Buy')
                print(predict_result, '\n')
                mt.shutdown()
                sleep(tf * 60)
            elif predict_result[-1] == 0:
                print('Sell')
                mt.shutdown()   
            else: 
                print('Can not predict')
            break
        else:
            print('Error')
            break

### Chamada das funções

In [ ]:
estrategia()
modelo()
conexao()
operacao()